In [ ]:
!pip install quantstats
!pip install catboost
!pip install ta -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 527.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import quantstats as qs
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import ta
import yfinance as yf
# Fetch historical daily data for QQQ
ticker = 'QQQ'
data = yf.download(ticker, start = '2013-01-01', end='2024-08-19')

# Data Preprocessing
data.dropna(inplace=True)
data.sort_index(inplace=True)
data['Returns'] = data['Adj Close'].pct_change()

def new_features(df):

    # Features related to price behavior
    df['high_low_ratio'] = df['High'] / df['Low']
    df['open_adjclose_ratio'] = df['Adj Close'] / df['Open']
    df['candle_to_wick_ratio'] = (df['Adj Close'] - df['Open']) / (df['High'] - df['Low'])
    upper_wick_size = df['High'] - df[['Open', 'Adj Close']].max(axis = 1)
    lower_wick_size = df[['Open', 'Adj Close']].min(axis = 1) - df['Low']
    df['upper_to_lower_wick_ratio'] = upper_wick_size /  lower_wick_size

    # Laggings of the Closing price
    df['lag1'] = df['Adj Close'].shift(1)
    df['lag2'] = df['Adj Close'].shift(2)
    df['lag3'] = df['Adj Close'].shift(3)
    df['lag4'] = df['Adj Close'].shift(4)
    df['lag5'] = df['Adj Close'].shift(5)

    # Close-to-laggings ratios
    df['close_to_lag1_ratio'] = df['Adj Close'] / df['lag1']
    df['close_to_lag2_ratio'] = df['Adj Close'] / df['lag2']
    df['close_to_lag3_ratio'] = df['Adj Close'] / df['lag3']
    df['close_to_lag4_ratio'] = df['Adj Close'] / df['lag4']
    df['close_to_lag5_ratio'] = df['Adj Close'] / df['lag5']

    # Moving averages
    df['ema5'] = ta.trend.ema_indicator(df['Adj Close'], window = 5)
    df['sma10'] = ta.trend.sma_indicator(df['Adj Close'], window = 10)

    # Price-to-moving average ratio
    df['close_ema5_ratio'] = df['Adj Close'] / df['ema5']
    df['close_sma10_ratio'] = df['Adj Close'] / df['sma10']

    # Ratio between shorter and longer Moving Averages
    df['ema5_sma10_ratio'] = df['ema5'] / df['sma10']

    # Volume-related features
    df['volume_sma5'] = ta.trend.sma_indicator(df['Volume'], window = 5)
    df['volume_sma10'] = ta.trend.sma_indicator(df['Volume'], window = 10)
    df['volume_shock'] = (df['Volume'] - df['volume_sma10']) / df['volume_sma10']
    df['volume_sma20_to_volume_sma50_ratio'] = df['volume_sma5'] / df['volume_sma10']
    df['volume_change'] = df['Volume'].pct_change()
    df['volume_price_trend'] = df['volume_change'] / df['Adj Close'].pct_change()

    # Volatility features
    df['10_days_volatility'] = df['Adj Close'].pct_change().rolling(window = 10).std()
    df['20_days_volatility'] = df['Adj Close'].pct_change().rolling(window = 20).std()
    df['9_to_20_day_vol_ratio'] = df['9_days_volatility'] / df['20_days_volatility']

    # Technical indicators
    df['rsi'] = ta.momentum.RSIIndicator(df['Adj Close']).rsi()
    df['rsi_overbought'] = (df['rsi'] >= 70).astype(int)
    df['rsi_oversold'] = (df['rsi'] <= 30).astype(int)
    df['cci'] = ta.trend.cci(df['High'], df['Low'], df['Adj Close'], window=10, constant=0.015)
    df['obv'] = ta.volume.OnBalanceVolumeIndicator(close=df['Adj Close'], volume=df['Volume']).on_balance_volume()
    df['obv_divergence_5_days'] = df['obv'].diff().rolling(10).sum() - df['Adj Close'].diff().rolling(5).sum()
    df['obv_divergence_10_days'] = df['obv'].diff().rolling(20).sum() - df['Adj Close'].diff().rolling(10).sum()
    df['ADX'] = ta.trend.ADXIndicator(df['High'], df['Low'], df['Adj Close'], window = 10).adx()
    df['ADI'] = ta.volume.AccDistIndexIndicator(df['High'], df['Low'], df['Adj Close'], df['Volume']).acc_dist_index()

    # Weekly returns
    df['weekly_returns'] = np.round(((df['Adj Close'] - df['Open']) / df['Open']) * 100,2)

    # Replacing infinite values by zeros
    df = df.replace([np.inf, -np.inf], 0)

    # Removing NaN values
    df = df.dropna()
    return df

# data = new_features(data)
# Feature Engineering
data['30_days_volatility'] = data['Adj Close'].pct_change().rolling(window = 30).std()
data['60_days_volatility'] = data['Adj Close'].pct_change().rolling(window = 60).std()
data['90_days_volatility'] = data['Adj Close'].pct_change().rolling(window = 90).std()
data['120_days_volatility'] = data['Adj Close'].pct_change().rolling(window = 120).std()
data['cci'] = ta.trend.cci(data['High'], data['Low'], data['Adj Close'], window=10, constant=0.015)
data['rsi'] = ta.momentum.RSIIndicator(data['Adj Close']).rsi()
data['obv'] = ta.volume.OnBalanceVolumeIndicator(close=data['Adj Close'], volume=data['Volume']).on_balance_volume()
data['SMA_7'] = data['Adj Close'].rolling(window=7).mean()
data['SMA_30'] = data['Adj Close'].rolling(window=30).mean()
data['SMA_90'] = data['Adj Close'].rolling(window=90).mean()
data['SMA_180'] = data['Adj Close'].rolling(window=180).mean()
data['SMA_400'] = data['Adj Close'].rolling(window=400).mean()
data['EMA_8'] = data['Close'].ewm(span=8, adjust=False).mean()
data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()
data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
data['EMA_100'] = data['Close'].ewm(span=100, adjust=False).mean()
data['EMA_250'] = data['Close'].ewm(span=250, adjust=False).mean()
data['EMA_500'] = data['Close'].ewm(span=500, adjust=False).mean()
data['close_to_ema8_ratio'] = data['Adj Close'] / data['EMA_8']

def calculate_rsi(data, window):
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_fibonacci_retracement(data, high_col='High', low_col='Low', close_col='Close'):
    """
    Calculate Fibonacci retracement levels for the given data.

    Parameters:
    data (pd.DataFrame): DataFrame containing stock price data with high, low, and close prices.
    high_col (str): Column name for the high prices. Default is 'High'.
    low_col (str): Column name for the low prices. Default is 'Low'.
    close_col (str): Column name for the close prices. Default is 'Close'.

    Returns:
    pd.DataFrame: DataFrame with added Fibonacci retracement levels.
    """
    # Identify the highest high and lowest low over a rolling window
    data['High_Max'] = data[high_col].rolling(window=14).max()
    data['Low_Min'] = data[low_col].rolling(window=14).min()

    # Calculate the Fibonacci retracement levels
    data['Fib_0.0'] = data['High_Max']
    data['Fib_0.236'] = data['High_Max'] - 0.236 * (data['High_Max'] - data['Low_Min'])
    data['Fib_0.382'] = data['High_Max'] - 0.382 * (data['High_Max'] - data['Low_Min'])
    data['Fib_0.5'] = data['High_Max'] - 0.5 * (data['High_Max'] - data['Low_Min'])
    data['Fib_0.618'] = data['High_Max'] - 0.618 * (data['High_Max'] - data['Low_Min'])
    data['Fib_0.764'] = data['High_Max'] - 0.764 * (data['High_Max'] - data['Low_Min'])
    data['Fib_1.0'] = data['Low_Min']

    # Drop temporary columns used for calculation
    data.drop(['High_Max', 'Low_Min'], axis=1, inplace=True)

    return data

# Add Fibonacci retracement levels to the data
data = calculate_fibonacci_retracement(data)

data['RSI'] = calculate_rsi(data, window=14)

def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    ema_short = data['Adj Close'].ewm(span=short_window, adjust=False).mean()
    ema_long = data['Adj Close'].ewm(span=long_window, adjust=False).mean()
    macd_line = ema_short - ema_long
    macd_signal = macd_line.ewm(span=signal_window, adjust=False).mean()
    return macd_line, macd_signal

data['MACD_Line'], data['MACD_Signal'] = calculate_macd(data)

def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    middle_band = data['Adj Close'].rolling(window=window).mean()
    rolling_std = data['Adj Close'].rolling(window=window).std()
    upper_band = middle_band + (rolling_std * num_std_dev)
    lower_band = middle_band - (rolling_std * num_std_dev)
    return middle_band, upper_band, lower_band

data['Bollinger_Middle'], data['Bollinger_Upper'], data['Bollinger_Lower'] = calculate_bollinger_bands(data)

def calculate_atr(data, window=14):
    data['Prev_Close'] = data['Adj Close'].shift(1)
    data['TR'] = data[['High', 'Low', 'Prev_Close']].apply(lambda x: max(x['High'] - x['Low'], abs(x['High'] - x['Prev_Close']), abs(x['Low'] - x['Prev_Close'])), axis=1)
    atr = data['TR'].rolling(window=window).mean()
    data.drop(['Prev_Close', 'TR'], axis=1, inplace=True)
    return atr

data['ATR'] = calculate_atr(data)

# Define target and features
data['target'] = np.where(data['Returns'].shift(-1) > 0, 1, 0)
data.dropna(inplace=True)

features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'SMA_7', 'SMA_30', 'SMA_90', 'SMA_180', 'SMA_400',
            'EMA_8', 'EMA_20',  'Fib_0.0', 'Fib_0.236','Fib_0.382','Fib_0.5','Fib_0.618', 'Fib_0.764',
            'Fib_1.0', 'RSI', 'MACD_Line', 'MACD_Signal', 'Bollinger_Middle', 'Bollinger_Upper', 'Bollinger_Lower', 'ATR']
features2 = [
    # 'high_low_ratio',
    # 'open_adjclose_ratio',
    # 'candle_to_wick_ratio',
    # 'upper_to_lower_wick_ratio',
    # 'lag1',
    # 'lag2',
    # 'lag3',
    # 'lag4',
    # 'lag5',
    # 'close_to_lag1_ratio',
    # 'close_to_lag2_ratio',
    # 'close_to_lag3_ratio',
    # 'close_to_lag4_ratio',
    # 'close_to_lag5_ratio',
    # 'ema5',
    # 'sma10',
    # 'close_ema5_ratio',
    # 'close_sma10_ratio',
    # 'ema5_sma10_ratio',
    # 'volume_sma5',
    # 'volume_sma10',
    # 'volume_shock',
    # 'volume_sma20_to_volume_sma50_ratio',
    # 'volume_change',
    # 'volume_price_trend',
    # '9_days_volatility',
    # '20_days_volatility',
    # '9_to_20_day_vol_ratio',
    # 'rsi',
    # 'rsi_overbought',
    # 'rsi_oversold',
    # 'cci',
    # 'obv',
    # 'obv_divergence_5_days',
    # 'obv_divergence_10_days',
    # 'ADX',
    # 'ADI',
    # 'weekly_returns'
]

features.extend(features2)

X = data[features]
# X = data.drop(['target', 'Returns'], axis=1)
y = data['target']
print(X.columns)
# Time-based split
split_date = '2018-01-01'
X_train = X[:split_date]
X_test = X[split_date:]
y_train = y[:split_date]
y_test = y[split_date:]

# Model Development
model = xgb.XGBClassifier(random_state=43)
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
print("Model Evaluation Report:")
# print(classification_report(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))

# Cross-validation
tscv = TimeSeriesSplit(n_splits=5)
cv_scores = cross_val_score(model, X, y, cv=tscv, scoring='accuracy')
print("Cross-validation Scores:")
print(cv_scores)
print("Mean Cross-validation Score:")
print(np.mean(cv_scores))

# Feature Importance
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print("Feature Importances:")
# print(feature_importance_df)
# features.extend(['Returns', 'Date'])
# data = data[features]

# Backtesting
data['Predicted_Signal'] = model.predict(X)
data['Strategy_Returns'] = data['Returns'] * data['Predicted_Signal'].shift(1)
data['Buy_and_Hold_Returns'] = data['Returns']

data['Date'] = data.reset_index()['Date']
if 'Date' in data.columns:
    data.rename(columns={'Date': 'Existing_Date'}, inplace=True)
# Reset index to make 'Date' a regular column
data = data.reset_index()

# Rename the new index column to 'Date'
data.rename(columns={'index': 'Date'}, inplace=True)

# Convert the 'date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])
# Remove timezone information
data['Date'] = data['Date'].dt.tz_localize(None)

# Define your date range
start_date = '2018-01-01'
end_date = '2024-08-19'

# Convert the start and end dates to datetime
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Filter the DataFrame based on the date range
data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Calculate cumulative returns
data['Strategy_Cumulative_Returns'] = (1 + data['Strategy_Returns']).cumprod() - 1
data['Buy_and_Hold_Cumulative_Returns'] = (1 + data['Buy_and_Hold_Returns']).cumprod() - 1

# Evaluate the strategy using quantstats
qs.extend_pandas()

# Calculate additional performance metrics
trades = data['Predicted_Signal'].diff().fillna(0)
entries = trades[trades == 1].index
exits = trades[trades == -1].index

if len(exits) > len(entries):
    exits = exits[:len(entries)]

trade_pairs = zip(entries, exits)
trade_profits = []
holding_periods = []
trade_table = pd.DataFrame()

for entry, exit in trade_pairs:
    entry_price = data.loc[entry, 'Adj Close']
    exit_price = data.loc[exit, 'Adj Close']
    new_trade = pd.DataFrame([{
        'Entry_Time': entry,
        'Exit_Time': exit,
        'Entry_Price': entry_price,
        'Exit_Price': exit_price
    }])

    # Append the new trade to trades_df using concat
    trade_table = pd.concat([trade_table, new_trade], ignore_index=True)
    trade_profits.append((exit_price - entry_price) / entry_price)
    holding_periods.append((exit - entry).days)

win_rate = sum(1 for profit in trade_profits if profit > 0) / len(trade_profits)
total_return = sum(trade_profits)
average_upside = np.mean([profit for profit in trade_profits if profit > 0])
average_downside = np.mean([profit for profit in trade_profits if profit < 0])
expected_pl_per_trade = np.mean(trade_profits)
max_loss = np.min(trade_profits)
max_drawdown = (data['Strategy_Cumulative_Returns'].cummax() - data['Strategy_Cumulative_Returns']).max()
max_runup = (data['Strategy_Cumulative_Returns'] - data['Strategy_Cumulative_Returns'].cummin()).max()
average_trading_days = np.mean(holding_periods)

# Print additional metrics
print("Trading Strategy Metrics:")
print("Win Rate: {:.2f}%".format(win_rate * 100))
print("Total Return: {:.2f}%".format(total_return * 100))
print("Average Upside: {:.2f}%".format(average_upside * 100))
print("Average Downside: {:.2f}%".format(average_downside * 100))
print("Expected P/L Per Trade: {:.2f}%".format(expected_pl_per_trade * 100))
print("Maximum Loss: {:.2f}%".format(max_loss * 100))
print("Maximum Drawdown: {:.2f}%".format(max_drawdown * 100))
print("Maximum Run-up: {:.2f}%".format(max_runup * 100))
print("Average Trading Days: {:.2f}".format(average_trading_days))

# Buy-and-Hold Metrics
buy_and_hold_value = (data['Adj Close'][-1] - data['Adj Close'][0]) / data['Adj Close'][0]
buy_and_hold_return = data['Buy_and_Hold_Cumulative_Returns'][-1]
buy_and_hold_max_drawdown = (data['Buy_and_Hold_Cumulative_Returns'].cummax() - data['Buy_and_Hold_Cumulative_Returns']).max()
buy_and_hold_max_runup = (data['Buy_and_Hold_Cumulative_Returns'] - data['Buy_and_Hold_Cumulative_Returns'].cummin()).max()

print("\n\nBuy-and-Hold Metrics:")
print("Total Return (Cumulative): {:.2f}%".format(buy_and_hold_return * 100))
print("Buy and Hold Value: {:.2f}%".format(buy_and_hold_value * 100))
print("Maximum Drawdown: {:.2f}%".format(buy_and_hold_max_drawdown * 100))
print("Maximum Run-up: {:.2f}%".format(buy_and_hold_max_runup * 100))

# Generate quantstats report
# qs.reports.full(data['Strategy_Cumulative_Returns'], benchmark=data['Buy_and_Hold_Cumulative_Returns'])

# print(trade_table)

[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'SMA_7', 'SMA_30',
       'SMA_90', 'SMA_180', 'SMA_400', 'EMA_8', 'EMA_20', 'Fib_0.0',
       'Fib_0.236', 'Fib_0.382', 'Fib_0.5', 'Fib_0.618', 'Fib_0.764',
       'Fib_1.0', 'RSI', 'MACD_Line', 'MACD_Signal', 'Bollinger_Middle',
       'Bollinger_Upper', 'Bollinger_Lower', 'ATR'],
      dtype='object')
Model Evaluation Report:
Confusion Matrix:
Cross-validation Scores:
[0.41805226 0.5368171  0.40142518 0.48456057 0.51543943]
Mean Cross-validation Score:
0.47125890736342047
Feature Importances:
Trading Strategy Metrics:
Win Rate: 75.61%
Total Return: 117.48%
Average Upside: 1.99%
Average Downside: -2.24%
Expected P/L Per Trade: 0.96%
Maximum Loss: -12.09%
Maximum Drawdown: 76.35%
Maximum Run-up: 220.87%
Average Trading Days: 13.44


Buy-and-Hold Metrics:
Total Return (Cumulative): 218.77%
Buy and Hold Value: 213.28%
Maximum Drawdown: 93.47%
Maximum Run-up: 244.61%


<ipython-input-4-3f5ef6e21f55>:362: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  buy_and_hold_value = (data['Adj Close'][-1] - data['Adj Close'][0]) / data['Adj Close'][0]
<ipython-input-4-3f5ef6e21f55>:363: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  buy_and_hold_return = data['Buy_and_Hold_Cumulative_Returns'][-1]


In [ ]:
trade_table.to_csv('trade_table_samuel_emmanuel.csv', index=False)

In [ ]:
trade_table

,Entry_Time,Exit_Time,Entry_Price,Exit_Price
0,2018-01-03,2018-01-04,153.104263,153.372147
1,2018-01-05,2018-01-09,154.912476,155.524750
2,2018-02-05,2018-03-06,151.276932,161.246002
3,2018-03-08,2018-03-12,162.508865,166.546234
4,2018-03-13,2018-03-19,164.278824,160.127701
...,...,...,...,...
118,2024-06-24,2024-06-28,473.959991,479.109985
119,2024-07-02,2024-07-03,486.980011,491.040009
120,2024-07-05,2024-07-09,496.160004,497.769989
121,2024-07-10,2024-07-16,502.959991,496.339996


In [ ]:
# # Generate quantstats report
# qs.reports.full(data['Strategy_Cumulative_Returns'], benchmark=data['Buy_and_Hold_Cumulative_Returns'])

# print(trade_table)